# Majorana Tapering 
### in the Stabilizer Subspace Projection formalism
Here, we take a look at the qubit reduction technique of [tapering](https://arxiv.org/abs/1701.08213) and an implementation based on the core `S3_projection` class. Unlike [Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456), this technique is *exact*, in the sense that it perfectly preserves the energy spectrum of the input operator.

At the core of qubit tapering is a symmetry of the Hamiltonian, which in this case means a set of universally commuting operators. The idea is that these operators must be simultaneously measureable and so can be treated independently of the remaining Hamiltonian terms. The method works by finding an independent generating set for the symmetry and seeks to find the 'correct' assignment of eigenvalues (called a *sector*), which completely determines the measurement outcome of the symmetry operators. Once this is obtained, the theory of stabilizers allows us to rotate the symmetry generators onto single Pauli $X$ operators, and since they must commute universally every operator of the rotated Hamiltonian will consist of an identity or Pauli $X$ in the corresponding qubit position. This means we can drop the qubit from the Hamiltonian, leaving in its place the eigenvalue determined by the chosen sector.

In [1]:
from symred.symplectic_form import PauliwordOp, QubitHamiltonian, MajoranaOp
import numpy as np
from openfermion import MajoranaOperator

In [2]:
operator1 = [
    [4],
]
coeffs1 = np.arange(2,len(operator1)+2)
M1 = MajoranaOp(operator1, coeffs1)
print(M1)
print()

operator2 = [
    [2],
    [3]
]
coeffs2 = 1*np.arange(5,len(operator2)+5)
M2 = MajoranaOp(operator2, coeffs2)
print(M2)
print()

M3 = M1 + M2

print(M3)
print()

M4 =M1 * M2
print(M4)
print()

M1_openf = M1.to_OF_op()
M2_openf = M2.to_OF_op()
print(M1_openf+M2_openf)
print()
print(M1_openf*M2_openf)

(2+0j) γ4

(5+0j) γ2 +
(6+0j) γ3

(2+0j) γ4 +
(6+0j) γ3 +
(5+0j) γ2

(-12+0j) γ3 γ4 +
(-10+0j) γ2 γ4

(5+0j) (2,) +
(6+0j) (3,) +
(2+0j) (4,)

(-10+0j) (2, 4) +
(-12+0j) (3, 4)


In [3]:
y1 = MajoranaOperator(term=(4,), coefficient=2)
y2 = MajoranaOperator(term=(2,), coefficient=5) + MajoranaOperator(term=(3,), coefficient=6)
y1*y2

MajoranaOperator.from_dict(terms={(2, 4): -10, (3, 4): -12})

In [4]:
operator = [
    [0,1,2,3],
    [5,6],
    [3,5],
    [0,1,2,3,4,5,6,7],
    [2,3],
    [4]
]

# operator = [
#     [0],
#     [1,2],
#     [3,4],
#     [0,1,2,3,4,5,6,7],
#     [2,3]
# ]

# operator = [
#     [1],
#     [2],
#     [3]

# ]

# # #
# operator = [
#     [0]
# ]


# operator = [
#     [1]
# ]

coeffs = np.arange(2,len(operator)+2)

###
M = MajoranaOp(operator, coeffs)
print(M)

M.adjacency_matrix()

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4


array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [5]:
print(M*M)
print()

C = M.to_OF_op()
out = C*C
print(out)
# print()
# print(C)

(17+0j) I +
(42+0j) γ4 γ5 γ6 +
(20+0j) γ4 γ5 γ6 γ7 +
(-56+0j) γ3 γ4 γ5 +
(36+0j) γ2 γ3 γ5 γ6 +
(84+0j) γ2 γ3 γ4 +
(-24+0j) γ0 γ1 +
(-60+0j) γ0 γ1 γ4 γ5 γ6 γ7 +
(40+0j) γ0 γ1 γ2 γ4 γ6 γ7 +
(12+0j) γ0 γ1 γ2 γ3 γ5 γ6 +
(28+0j) γ0 γ1 γ2 γ3 γ4 +
(-30+0j) γ0 γ1 γ2 γ3 γ4 γ7

(17+0j) () +
(-24+0j) (0, 1) +
(28+0j) (0, 1, 2, 3, 4) +
(-30+0j) (0, 1, 2, 3, 4, 7) +
(12+0j) (0, 1, 2, 3, 5, 6) +
(40+0j) (0, 1, 2, 4, 6, 7) +
(-60+0j) (0, 1, 4, 5, 6, 7) +
(84+0j) (2, 3, 4) +
(36+0j) (2, 3, 5, 6) +
(-56+0j) (3, 4, 5) +
(42+0j) (4, 5, 6) +
(20+0j) (4, 5, 6, 7)


In [6]:
M.adjacency_matrix()

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

# get basis for operator

In [7]:
from symred.S3_projection import gf2_basis_for_gf2_rref, gf2_gaus_elim
ZX_symp = M.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0]])

In [8]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6


In [9]:
print(M)
print()
print(basis_op)

openF_M_op = M.to_OF_op()
basis_op_openF =  basis_op.to_OF_op()
print('commmutes: ', openF_M_op*basis_op_openF == basis_op_openF*openF_M_op)

print(M.commutes_termwise(basis_op))
M.commutes(basis_op)

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6
commmutes:  True
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


True

In [33]:
0.5*1j

0.5j

In [77]:
majorana = np.zeros((2*M_terms, N_sites))
sign = {0:1j, 1:-1j}
for maj_ind, operator in enumerate(ham.terms.keys()):
    for index, crea_flag in operator:
        majorana[maj_ind, index]=1
        majorana[2*maj_ind+1, index]=1
        
    print()
majorana

array([[1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 1., 1., 1., 0., 1.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 1.]])

In [138]:

    

ham = (FermionOperator('0^ 3', .5) +
       FermionOperator('3^ 0', 0.5) +
      FermionOperator('3^ 2^ 0 1 5^', 0.5))

M_out = convert_openF_fermionic_op_to_maj_op(ham)

M_out.to_OF_op() == get_majorana_operator(ham)

True